In [1]:
import pandas as pd

import param

In [4]:
#!ls -lath /cellar/users/btsui/Data/SRA/DUMP/sra_dump.csv.gz

In [ ]:
%time sra_dump_df=pd.read_csv(param.full_meta_dir)

In [72]:
#sra_dump_df['']

#### check if data is in SRA

In [6]:
import os
import numpy as np


In [7]:
#!ls -alth /nrnb/users/btsui/Data/all_seq/rnaseq/ > .ls.txt

In [8]:
allProcessedFnames=pd.Series(os.listdir(param.count_out_dir),dtype=np.str)

In [9]:
postfix_m=allProcessedFnames.str.contains('.abundance.tsv.gz$')
processedRnaseqSrr=allProcessedFnames[postfix_m].str.split('.').str[0].unique()

In [10]:
#processedRnaseqSrr

In [11]:
len(processedRnaseqSrr)

454901

In [49]:
sra_dump_df['Skymap_TranscriptCount_Processed']=sra_dump_df['Run'].isin(processedRnaseqSrr)

In [50]:
m_processed=sra_dump_df['Skymap_TranscriptCount_Processed']

### select the species

In [51]:
selectedSpecies='Danio_rerio'

In [52]:
faBaseDir='/cellar/users/btsui/Data/ensembl/release/cdna/'

In [53]:
from tqdm import tqdm

In [5]:
#sra_dump_df['new_ScientificName']

In [66]:
m_specie=sra_dump_df['new_ScientificName']==selectedSpecies

In [67]:
m_specie.sum()

0

In [55]:
m=m_processed&m_specie

In [56]:
sra_dump_df_sub=sra_dump_df[m]

sra_dump_df_in=sra_dump_df_sub.head(n=10)

In [63]:
m_specie.sum()

0

In [57]:
nTotal=sra_dump_df_sub.shape[0]

In [58]:
print ('# of samples to merge:',nTotal)

# of samples to merge: 0


In [59]:
#inSrr='DRR050162'

In [60]:
from tqdm import tqdm
from multiprocessing import Pool

In [47]:
nProcess=32
inSrrs=sra_dump_df_in['Run'].unique()

In [48]:
def parseOne(inSrr):
    abundanceDir=param.count_out_dir+'{}.abundance.tsv.gz'.format(inSrr)
    tmpDf=pd.read_csv(abundanceDir,sep='\t')
    tmpDf2=tmpDf.set_index('target_id')[['est_counts','tpm']].astype(float)
    return tmpDf2

In [33]:
#03:57 for merging 19034 samples 
with Pool(nProcess) as p:
    myL=list(tqdm( p.imap(parseOne,inSrrs),total=nTotal))

0it [00:00, ?it/s]


In [34]:
%%time
mergedDf=pd.concat(myL,axis=1,keys=inSrrs)

ValueError: No objects to concatenate

# export data

In [ ]:
outputDir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged/{}.pickle'.format(selectedSpecies)

In [2]:
mergedDf.dtypes

NameError: name 'mergedDf' is not defined

In [ ]:
##find samples with data 
geneWithDataMask=(mergedDf.sum(axis=1))>0
sampleWithDataMask=(mergedDf.sum(axis=0))>0

In [ ]:
geneWithDataMask.mean(),sampleWithDataMask.mean()

In [ ]:
mergedDf.loc[]

In [ ]:
export_m=withDataMask

In [189]:
mergedDf.to_pickle(outputDir)

In [218]:
!ls -lath /nrnb/users/btsui/Data/all_seq/rnaseq_merged/

total 9.4M
-rw-r--r--  1 btsui users 9.2M Aug 28 17:27 Danio_rerio.pickle
drwxr-xr-x  2 btsui users  512 Aug 28 17:27 .
drwxr-xr-x 16 btsui users 128K Aug 28 17:26 ..


In [185]:
#!mkdir /nrnb/users/btsui/Data/all_seq/rnaseq_merged/